In [1]:
import numpy as np
import pandas as pd
from spacepy import coordinates as coord
from spacepy.time import Ticktock

In [2]:
def geosph2geocar(times:np.ndarray,alts:np.ndarray,lats:np.ndarray,lons:np.ndarray):
    """
    for points
    Args:
        times: element is np.datetime64
        alts: unit is Re
        lats: degree
        lons: ~

    Returns:
        'geo,car' Coords instance
    """
    arr_n_3 = np.column_stack((alts, lats, lons))
    c = coord.Coords(arr_n_3,"GEO", "sph", ticks=Ticktock(times, "ISO"))
    return c.convert("GEO","car")

In [6]:
Re = 6371.2

# trace
file_path = r"G:\master\pyaw\scripts\results\aw_cases\archive\trace_points\end_trace_points_SwarmA_12728_case_20160301T014840_20160301T014900.pkl"
df_trace = pd.read_pickle(file_path)
df_trace_np = df_trace.values
df_trace_np_alt_re = df_trace_np.copy()
df_trace_np_alt_re[:, 0] = df_trace_np_alt_re[:, 0] + Re
df_trace_np_alt_re[:, 0] = df_trace_np_alt_re[:, 0] / Re
time_trace = np.datetime64("2016-03-01T01:49:00")  # the corresponding trace time
times_trace = np.array([str(time_trace) for _ in range(len(df_trace_np_alt_re[:, 0]))])
geo_car_trace = geosph2geocar(times_trace,df_trace_np_alt_re[:,0],df_trace_np_alt_re[:,1],df_trace_np_alt_re[:,2])

# satellite
satellite_path = r"V:\aw\swarm\vires\measurements\SW_EXPT_EFIA_TCT16\SW_EXPT_EFIA_TCT16_12728_20160301T012924_20160301T030258.pkl"
df_sa = pd.read_pickle(satellite_path)
swarm_radius = 6816838.5 * 1e-3  # km
swarm_altitude = swarm_radius - Re  # km（初始高度）
lats_sa = df_sa["Latitude"].values
lons_sa = df_sa["Longitude"].values
# km convert to Re
alts_sa = np.full(len(lats_sa),swarm_altitude)
alts_sa = alts_sa + Re
alts_sa = alts_sa / Re
times_sa = df_sa.index.values
times_sa_str = np.array([str(t) for t in times_sa])
geo_car_sa = geosph2geocar(times_sa_str,alts_sa,lats_sa,lons_sa)

In [30]:
# c = coord.Coords(df_trace_np_alt_re, "GEO", "sph", ticks=Ticktock([str(time_trace) for _ in range(len(df_trace_np_alt_re[:, 0]))], "ISO"))

In [31]:
# c_geo_car = c.convert("GEO","car")  # the unit is 'Re'

In [ ]:
# c = coord.Coords(
#     [(alt0 + Re) / Re, lat0, lon0], "GEO", "sph", ticks=Ticktock(str(time), "ISO")
# )
# # convert geo2gsm
# c_gsm = c.convert("GSM", "car")
# ut = time.astype("datetime64[s]").astype(np.int64)